In [1]:
import os
import requests
import json
import pandas as pd
import numpy as np

In [2]:
!git clone https://github.com/ryansherby/BLS-Transformer.git

fatal: destination path 'BLS-Transformer' already exists and is not an empty directory.


In [3]:
!pip install bls-transformer

In [4]:
# Add the path to the directory containing the file to the system path
import sys
sys.path.append('/Users/christina___99/BLS-Transformer/src/bls_transformer')

# Import the class from the file
from BLS import API

In [5]:
df_codes = pd.read_csv('la.series.txt', delimiter='\t')

In [6]:
df_codes.head()

,series_id,area_type_code,area_code,measure_code,seasonal,srd_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,LASBS060000000000003,L,BS0600000000000,3,S,6,"Unemployment Rate: Balance of California, stat...",NaN,1976,M01,2024,M03
1,LASBS060000000000004,L,BS0600000000000,4,S,6,"Unemployment: Balance of California, state les...",NaN,1976,M01,2024,M03
2,LASBS060000000000005,L,BS0600000000000,5,S,6,"Employment: Balance of California, state less ...",NaN,1976,M01,2024,M03
3,LASBS060000000000006,L,BS0600000000000,6,S,6,"Labor Force: Balance of California, state less...",NaN,1976,M01,2024,M03
4,LASBS120000000000003,L,BS1200000000000,3,S,12,"Unemployment Rate: Balance of Florida, state l...",NaN,1990,M01,2024,M03


In [7]:
ur_series_ids = []

if ((df_codes['area_type_code'] == 'F') & (df_codes['measure_code'] == 3)).any():
    ur_series_ids.extend(df_codes.loc[df_codes['area_type_code'] == 'F', 'series_id                     '].tolist())

In [8]:
df_codes = df_codes[(df_codes['area_type_code']=='F') & (df_codes['measure_code']==3)]

In [9]:
df_codes.columns

Index(['series_id                     ', 'area_type_code', 'area_code',
       'measure_code', 'seasonal', 'srd_code', 'series_title',
       'footnote_codes', 'begin_year', 'begin_period', 'end_year',
       'end_period'],
      dtype='object')

In [10]:
ur_series_ids = df_codes['series_id                     '].tolist()

In [11]:
len(ur_series_ids)

3224

In [12]:
ur_series_ids = [s[:20] for s in ur_series_ids]

In [13]:
ur_seriesIDs_usa = ur_series_ids[0:3146]
ur_seriesIDs_pr = ur_series_ids[3146:]

In [14]:
chunk_usa = [ur_seriesIDs_usa[i:i + 50] for i in range(0, len(ur_seriesIDs_usa), 50)]
chunk_pr = [ur_seriesIDs_pr[i:i + 50] for i in range(0, len(ur_seriesIDs_pr), 50)]

In [15]:
print(len(ur_seriesIDs_usa), 'USA Counties in', len(chunk_usa), 'iterations')
print(len(ur_seriesIDs_pr), 'PR Counties in', len(chunk_pr), 'iterations')

3146 USA Counties in 63 iterations
78 PR Counties in 2 iterations


Response Message: No Data Available for Series LAUCN022010000000003 Year: 2020

Response Message: No Data Available for Series LAUCN022010000000003 Year: 2021

Response Message: No Data Available for Series LAUCN022010000000003 Year: 2022

Response Message: No Data Available for Series LAUCN022320000000003 Year: 2020

In [16]:
ur_data_usa_dict = {}

for i in range(0, 63):
    ur_seriesIDs_usa_i = chunk_usa[i]
    ur_i = API(series_id=ur_seriesIDs_usa_i, registration_key='dc8fb679ad0748d89af62c597a479cff', start_year='2020', end_year='2022')
    ur_i.transform_json()
    ur_data_i = ur_i.get_DataFrame(id=ur_seriesIDs_usa_i, catalog_value='series_title', axis=0)
    
    # Store the data frame in the dictionary with key 'unemployment_data_i'
    ur_data_usa_dict[f'ur_data_{i}'] = ur_data_i

Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.
Request Status: REQUEST_SUCCEEDED
Response Message: No Data Available for Series LAUCN022010000000003 Year: 2020
Response Message: No Data Available for Series LAUCN022010000000003 Year: 2021
Response Message: No Data Available for Series LAUCN022010000000003 Year: 2022
Response Message: No Data Available for Series LAUCN022320000000003 Year: 2020
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.
Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.
Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.
Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.
Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its i

In [17]:
ur_usa_list = list(ur_data_usa_dict.values())
ur_usa_data = pd.concat(ur_usa_list)
ur_usa_data = ur_usa_data.drop(columns=['Year', 'Period'])
ur_usa_data = ur_usa_data.reset_index()
ur_usa_data = ur_usa_data.rename(columns={'level_0':'County'})

In [18]:
ur_usa_data.head()

,County,Year,Period,Period Name,Value
0,"Unemployment Rate: Autauga County, AL (U)",2022,12,December,1.8
1,"Unemployment Rate: Autauga County, AL (U)",2022,11,November,1.9
2,"Unemployment Rate: Autauga County, AL (U)",2022,10,October,2.1
3,"Unemployment Rate: Autauga County, AL (U)",2022,9,September,2.2
4,"Unemployment Rate: Autauga County, AL (U)",2022,8,August,2.5


In [20]:
ur_seriesIDs_usa_1 = chunk_usa[0]
ur_usa_1 = API(series_id=ur_seriesIDs_usa_1, registration_key='dc8fb679ad0748d89af62c597a479cff', start_year='2020', end_year='2022')
ur_usa_1.transform_json()
ur_usa_1 = ur_usa_1.get_DataFrame(id=ur_seriesIDs_usa_1, catalog_value='series_title', axis=0)    

Request Status: REQUEST_SUCCEEDED
Success! Use get_DataFrame() to access one or more DataFrames by its index or id.


In [22]:
ur_usa_1

Year Period  \
                                          Year Period                
Unemployment Rate: Autauga County, AL (U) 2022 12      2022     12   
                                               11      2022     11   
                                               10      2022     10   
                                               9       2022      9   
                                               8       2022      8   
...                                                     ...    ...   
Unemployment Rate: Monroe County, AL (U)  2020 5       2020      5   
                                               4       2020      4   
                                               3       2020      3   
                                               2       2020      2   
                                               1       2020      1   

                                                      Period Name Value  
                                          Year Period                    
Unemployment Rate: Autauga County, AL (U) 2022 12        December   1.8  
                                               11        November   1.9  
                                               10         October   2.1  
                                               9        September   2.2  
                                               8           August   2.5  
...                                                           ...   ...  
Unemployment Rate: Monroe County, AL (U)  2020 5              May  11.9  
                                               4            April  15.3  
                                               3            March   5.9  
                                               2         February   5.7  
                                               1          January   5.8  

[1800 rows x 4 columns]

In [23]:
ur_usa_data.to_csv('ur_usa_data.csv', index=False)

In [25]:
ur_seriesIDs_pr_1 = chunk_pr[0]
ur_pr_1 = API(series_id=ur_seriesIDs_pr_1, registration_key='dc8fb679ad0748d89af62c597a479cff', start_year='2020', end_year='2022')
ur_pr_1.json_response

Request Status: REQUEST_SUCCEEDED


{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 1364,
 'message': [],
 'Results': {'series': [{'seriesID': 'LAUCN720010000000003',
    'catalog': {'series_title': 'Unemployment Rate: Adjuntas Municipio, PR (U)',
     'series_id': 'LAUCN720010000000003',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'Local Area Unemployment Statistics',
     'survey_abbreviation': 'LA',
     'measure_data_type': 'unemployment rate',
     'area': 'Adjuntas Municipio, PR',
     'area_type': 'Counties and equivalents'},
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '9.1',
      'footnotes': [{'code': 'T',
        'text': 'Data were subject to revision on April 19, 2024.'}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '7.7',
      'footnotes': [{'code': 'T',
        'text': 'Data were subject to revision on April 19, 2024.'}]},
     {'year': '2022',
      'period': 'M10',
      'perio

In [26]:
seriesTitle = []
year = []
period = []
periodName = []
value = []

for i in range(0, 50):
    
    series_title = ur_pr_1.json_response['Results']['series'][i]['catalog']['series_title']
    seriesTitle.append(series_title)
    

    for data_dict in ur_pr_1.json_response['Results']['series'][i]['data']:
        year.append(int(data_dict['year']))
        period.append(int(data_dict['period'][1:]))
        periodName.append(str(data_dict['periodName']))

        # Check if the value is a dash ("-")
        if data_dict['value'] == '-':
            value.append(None)  # Set value to None if it's a dash
        else:
            try:
                value.append(float(data_dict['value']))  # Otherwise, convert value to float
            except ValueError:
                print("Error converting value to float:", data_dict['value'])

    json_list = [year, period, periodName, value]
    ur_pr_1_data = pd.DataFrame(json_list).transpose()

In [27]:
counties_1 = []
for i in range(0,50):
    titles = [seriesTitle[i]] * 36
    counties_1.extend(titles)

len(counties_1)

1800

In [28]:
ur_pr_1_data['County'] = pd.DataFrame(counties_1)
col = ur_pr_1_data.pop('County')
ur_pr_1_data.insert(0, col.name, col)
ur_pr_1_data

,County,0,1,2,3
0,"Unemployment Rate: Adjuntas Municipio, PR (U)",2022,12,December,9.1
1,"Unemployment Rate: Adjuntas Municipio, PR (U)",2022,11,November,7.7
2,"Unemployment Rate: Adjuntas Municipio, PR (U)",2022,10,October,8.0
3,"Unemployment Rate: Adjuntas Municipio, PR (U)",2022,9,September,7.3
4,"Unemployment Rate: Adjuntas Municipio, PR (U)",2022,8,August,7.3
...,...,...,...,...,...
1795,"Unemployment Rate: Mayaguez Municipio, PR (U)",2020,5,May,10.1
1796,"Unemployment Rate: Mayaguez Municipio, PR (U)",2020,4,April,None
1797,"Unemployment Rate: Mayaguez Municipio, PR (U)",2020,3,March,None
1798,"Unemployment Rate: Mayaguez Municipio, PR (U)",2020,2,February,9.6


In [31]:
ur_seriesIDs_pr_2 = chunk_pr[1]
ur_pr_2 = API(series_id=ur_seriesIDs_pr_2, registration_key='dc8fb679ad0748d89af62c597a479cff', start_year='2020', end_year='2022')
ur_pr_2.json_response

Request Status: REQUEST_SUCCEEDED


{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 893,
 'message': [],
 'Results': {'series': [{'seriesID': 'LAUCN720990000000003',
    'catalog': {'series_title': 'Unemployment Rate: Moca Municipio, PR (U)',
     'series_id': 'LAUCN720990000000003',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'Local Area Unemployment Statistics',
     'survey_abbreviation': 'LA',
     'measure_data_type': 'unemployment rate',
     'area': 'Moca Municipio, PR',
     'area_type': 'Counties and equivalents'},
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '8.4',
      'footnotes': [{'code': 'T',
        'text': 'Data were subject to revision on April 19, 2024.'}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '9.0',
      'footnotes': [{'code': 'T',
        'text': 'Data were subject to revision on April 19, 2024.'}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': '

In [32]:
seriesTitle = []
year = []
period = []
periodName = []
value = []

for i in range(0, 28):
    
    series_title = ur_pr_2.json_response['Results']['series'][i]['catalog']['series_title']
    seriesTitle.append(series_title)
    

    for data_dict in ur_pr_2.json_response['Results']['series'][i]['data']:
        year.append(int(data_dict['year']))
        period.append(int(data_dict['period'][1:]))
        periodName.append(str(data_dict['periodName']))

        # Check if the value is a dash ("-")
        if data_dict['value'] == '-':
            value.append(None)  # Set value to None if it's a dash
        else:
            try:
                value.append(float(data_dict['value']))  # Otherwise, convert value to float
            except ValueError:
                print("Error converting value to float:", data_dict['value'])

    json_list = [year, period, periodName, value]
    ur_pr_2_data = pd.DataFrame(json_list).transpose()

In [33]:
counties_2 = []
for i in range(0,28):
    titles = [seriesTitle[i]] * 36
    counties_2.extend(titles)
    
len(counties_2)

1008

In [35]:
ur_pr_2_data['County'] = pd.DataFrame(counties_2)
col = ur_pr_2_data.pop('County')
ur_pr_2_data.insert(0, col.name, col)
ur_pr_2_data

,County,0,1,2,3
0,"Unemployment Rate: Moca Municipio, PR (U)",2022,12,December,8.4
1,"Unemployment Rate: Moca Municipio, PR (U)",2022,11,November,9.0
2,"Unemployment Rate: Moca Municipio, PR (U)",2022,10,October,9.2
3,"Unemployment Rate: Moca Municipio, PR (U)",2022,9,September,8.6
4,"Unemployment Rate: Moca Municipio, PR (U)",2022,8,August,8.1
...,...,...,...,...,...
1003,"Unemployment Rate: Yauco Municipio, PR (U)",2020,5,May,12.0
1004,"Unemployment Rate: Yauco Municipio, PR (U)",2020,4,April,None
1005,"Unemployment Rate: Yauco Municipio, PR (U)",2020,3,March,None
1006,"Unemployment Rate: Yauco Municipio, PR (U)",2020,2,February,17.4


In [36]:
ur_pr_data = pd.concat([ur_pr_1_data, ur_pr_2_data])

In [37]:
ur_pr_data = ur_pr_data.rename(columns={0:'Year', 1:'Period', 2:'Period Name', 3:'Value'})

In [38]:
ur_pr_data.to_csv('ur_pr_data.csv')

In [39]:
unemployment_rate_monthly_data = pd.concat([ur_usa_data, ur_pr_data])

In [40]:
unemployment_rate_monthly_data['Value'] = unemployment_rate_monthly_data['Value'].astype('float64')
unemployment_rate_monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115920 entries, 0 to 1007
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   County       115920 non-null  object 
 1   Year         115920 non-null  object 
 2   Period       115920 non-null  object 
 3   Period Name  115920 non-null  object 
 4   Value        115764 non-null  float64
dtypes: float64(1), object(4)
memory usage: 5.3+ MB


In [42]:
df_codes.columns

Index(['series_id', 'area_type_code', 'area_code', 'measure_code', 'seasonal',
       'srd_code', 'County', 'footnote_codes', 'begin_year', 'begin_period',
       'end_year', 'end_period'],
      dtype='object')

In [41]:
df_codes = df_codes.rename(columns={'series_id                     ':'series_id', 'series_title':'County'})

In [43]:
df_codes_copy = df_codes[['series_id', 'area_type_code', 'area_code', 'measure_code',
                          'seasonal', 'srd_code', 'County']].copy()

In [44]:
unemployment_rate_monthly_data = pd.merge(df_codes_copy, unemployment_rate_monthly_data,
                                          how='left', on=['County'])

In [48]:
unemployment_rate_monthly_data = unemployment_rate_monthly_data.rename(columns={'Value':'Unemployment Rate'})

In [49]:
unemployment_rate_monthly_data.to_csv('unemployment_rate_monthly_data.csv', index=False)